### Andrew Burton

In [19]:
import json
import dash
import pickle
import numpy as np
import pandas as pd
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
import dash_core_components as dcc
from urllib.request import urlopen
import dash_html_components as html
import dash_bootstrap_components as dbc
from plotly.subplots import make_subplots
from dash.dependencies import Input, Output, State

# Data Sources
#### Hospital Database
https://hifld-geoplatform.opendata.arcgis.com/datasets/6ac5e325468c4cb9b905f1728d6fbf0f_0/data?geometry=70.851%2C-16.829%2C-101.766%2C72.120
#### Census Demographic Database
https://www.census.gov/data/datasets/2010/demo/popest/modified-race-data-2010.html
#### Life Expectancy Dataset
https://www.cdc.gov/nchs/nvss/usaleep/usaleep.html
#### Income Dataset
https://apps.bea.gov/iTable/iTable.cfm?reqid=70&step=1&acrdn=6


# Data

In [2]:
# Dataframes
H = pd.read_pickle('H.pkl')
S = pd.read_pickle('S.pkl')

hospitals = pd.read_pickle('hospitals.pkl')
hospitals = hospitals[hospitals.status=='OPEN'].to_crs('EPSG:4326')

# json of counties
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as r:
    counties_json = json.load(r)

# Labels/Groupings/Columns

In [3]:
# Labels n lists
state_dict = {'01':['Alabama','AL','Southeast'],'02':['Alaska','AK','West'],'04':['Arizona','AZ','Southwest'],
              '05':['Arkansas','AR','Southeast'],'06':['California','CA','West'],'08':['Colorado','CO','West'],
              '09':['Connecticut','CT','Northeast'],'10':['Delaware','DE','Northeast'],
              '11':['District of Columbia','DC','Northeast'],'12':['Florida','FL','Southeast'],
              '13':['Georgia','GA','Southeast'],'15':['Hawaii','HI','West'],'16':['Idaho','ID','West'],
              '17':['Illinois','IL','Midwest'],'18':['Indiana','IN','Midwest'],'19':['Iowa','IA','Midwest'],
              '20':['Kansas','KS','Midwest'],'21':['Kentucky','KY','Southeast'],
              '22':['Louisiana','LA','Southeast'],'23':['Maine','ME','Northeast'],
              '24':['Maryland','MD','Northeast'],'25':['Massachusetts','MA','Northeast'],
              '26':['Michigan','MI','Midwest'],'27':['Minnesota','MN','Midwest'],
              '28':['Mississippi','MS','Southeast'],'29':['Missouri','MO','Midwest'],'30':['Montana','MT','West'],
              '31':['Nebraska','NE','Midwest'],'32':['Nevada','NV','West'],
              '33':['New Hampshire','NH','Northeast'],'34':['New Jersey','NJ','Northeast'],
              '35':['New Mexico','NM','Southwest'],'36':['New York','NY','Northeast'],
              '37':['North Carolina','NC','Southeast'],'38':['North Dakota','ND','Midwest'],
              '39':['Ohio','OH','Midwest'],'40':['Oklahoma','OK','Southwest'],'41':['Oregon','OR','West'],
              '42':['Pennsylvania','PA','Northeast'],'44':['Rhode Island','RI','Northeast'],
              '45':['South Carolina','SC','Southeast'],'46':['South Dakota','SD','Midwest'],
              '47':['Tennessee','TN','Southeast'],'48':['Texas','TX','Southwest'],'49':['Utah','UT','West'],
              '50':['Vermont','VT','Northeast'],'51':['Virginia','VA','Southeast'],
              '53':['Washington','WA','West'],'54':['West Virginia','WV','Southeast'],
              '55':['Wisconsin','WI','Midwest'],'56':['Wyoming','WY','West']}

races = ['Total Population','Multiracial','Native Hawaiian/Pacific Islander',
         'American/Alaskan Native','Asian','Black','White','Hispanic']
states = [s[0] for s in state_dict.values()]
regions = ['West','Southwest','Midwest','Southeast','Northeast']
alljurs = ['US'] + regions + states        
hospital_types = ['All Facilities'] + list(pd.Series(hospitals['type'].unique()).str.title())

map_col = races + hospital_types +\
          [f'Percentage {r}' for r in races[1:]] + \
          [f'{r} Population per Square km' for r in races] +\
          ['All Facilities per 100k']+\
          [f'{h} Facilities per 100k' for h in hospital_types[1:]] +\
          ['All Facilities per Square km']+\
          [f'{h} Facilities per Square km' for h in hospital_types[1:]] +\
          ['Average Income','Average Life Expectancy'] 

bea_url = 'https://apps.bea.gov/iTable/iTable.cfm?reqid=70&step=1&acrdn=6'
cdc_url = 'https://www.cdc.gov/nchs/nvss/usaleep/usaleep.html'
hifld_url = 'https://hifld-geoplatform.opendata.arcgis.com/datasets/6ac5e325468c4cb9b905f1728d6fbf0f_0/explore?location=8.136124%2C-15.457900%2C2.94&showTable=true'
census_url = 'https://www.census.gov/data/datasets/2010/demo/popest/modified-race-data-2010.html'


# Options

In [17]:
# Dropdowns

# Choropleth dropdown (with sublists)
dem_count = [dbc.DropdownMenuItem(i,id=f"{i.replace(' ','_').replace('/','+')}2") for i in map_col[:8]]
dem_pct   = [dbc.DropdownMenuItem(i,id=f"{i.replace(' ','_').replace('/','+')}2") for i in map_col[19:26]]
dem_area  = [dbc.DropdownMenuItem(i,id=f"{i.replace(' ','_').replace('/','+')}2") for i in  map_col[26:34]]

demographics = [dbc.DropdownMenu(dem_count,label='Count'),
                dbc.DropdownMenuItem(divider=True),
                dbc.DropdownMenu(dem_pct,label='Percent'),
                dbc.DropdownMenuItem(divider=True),
                dbc.DropdownMenu(dem_area,label='Density')]

health_count = [dbc.DropdownMenuItem(i,id=f"{i.replace(' ','_').replace('/','+')}2") for i in map_col[8:19]]
health_capit = [dbc.DropdownMenuItem(i,id=f"{i.replace(' ','_').replace('/','+')}2") for i in map_col[34:45]]
health_area  = [dbc.DropdownMenuItem(i,id=f"{i.replace(' ','_').replace('/','+')}2") for i in  map_col[45:56]]

healthcare = [dbc.DropdownMenu(health_count,label='Count'),
              dbc.DropdownMenuItem(divider=True),
              dbc.DropdownMenu(health_capit,label='Percent'),
              dbc.DropdownMenuItem(divider=True),
              dbc.DropdownMenu(health_area,label='Density')]

choptions = [dbc.DropdownMenuItem('Average Income',id='Average_Income2'),
             dbc.DropdownMenuItem('Average Life Expectancy',id='Average_Life_Expectancy2'),
             dbc.DropdownMenuItem(divider=True),
             dbc.DropdownMenu(demographics,label='Demographics'),
             dbc.DropdownMenuItem(divider=True),
             dbc.DropdownMenu(healthcare,label='Healthcare')]

# Demographic dropdown
doptions = [dbc.DropdownMenuItem(r,id=f"{r.replace(' ','_').replace('/','+')}3") for r in races]

# Facility dropdown
hoptions = [dbc.DropdownMenuItem(h,id=f"{h.replace(' ','_').replace('/','+')}4") for h in hospital_types]

# Jurisdiction dropdown (with sublists)
stoptions = [dbc.DropdownMenuItem(s,id=f"{s.replace(' ','_').replace('/','+')}5") for s in states]
roptions  = [dbc.DropdownMenuItem(r,id=f"{r}5") for r in regions]

joptions = [dbc.DropdownMenuItem('US',id='US5'),
            dbc.DropdownMenuItem(divider=True),
            dbc.DropdownMenu(roptions,label='Regions'),
            dbc.DropdownMenuItem(divider=True),
            dbc.DropdownMenu(stoptions,label='States')]

In [5]:
# Radio and checklist options
pct_active =    [{'label':' Tot','value':''},
                 {'label':' Pct','value':'Percentage '},
                 {'label':' Dens','value':' Population per Square km'}]
pct_disabled =  [{'label':' Tot','value':''},
                 {'label':' Pct','value':'Percentage ','disabled':True},
                 {'label':' Dens','value':' Population per Square km'}]
rad_foptions =  [{'label':' Tot\t','value':''},
                 {'label':' Per 100k','value':' Facilities per 100k'},
                 {'label':' Dens','value':' Facilities per Square km'}]
rad_joptions =  [{'label':' Scale by County Pop','value':'scale'}]
rad_choptions = [{'label':' Show Facilities','value':'show'}]

In [18]:
# Methods for creating dropdowns/radios/checklists
def create_dropdown(label,droptions,html_id):
    '''Returns column with dropdown and display of selected value with input label, droptions, and html_id'''
    return dbc.Col([
               dbc.Row([
                   dbc.DropdownMenu(label=label,
                                    children=droptions),
                   html.H6(id=html_id,
                           style={'text-align':'center',
                                  'margin-left':'5px',
                                  'margin-right':'5px'})])],
                   width={'size':3})

def create_radio(radio_id,roptions):
    '''Returns column with radio selector and display of selected value with input label and roptions'''
    return dbc.Col([
               dcc.RadioItems(id=radio_id, 
                              options=roptions,
                              inputStyle={"margin-left":"7px"},
                              value='')],
               width={'size':3})

def create_checklist(checklist_id,chloptions):
    '''Returns column with checklist and display of selected value with input label, droptions, and html_id'''
    return dbc.Col([# Show hospitals
               dcc.Checklist(id=checklist_id,
                             options=chloptions,
                             value=[''])],
               width={'size':3})

# Layout

In [7]:
# Header
header = dbc.Row([# Header and tabs
                  dbc.Col([# Header
                      html.H1('Healthcare Access')],
                      width=5,
                      style={'padding':'0px 50px'}),
                  dbc.Col([# Tabs
                      dbc.Nav([dbc.NavLink('State Comparisons',
                                            href='/',
                                            active='exact'),
                               dbc.NavLink('County Comparisons',
                                            href='/county',
                                            active='exact'),
                               dbc.NavLink('Data Sources',
                                            href='/data',
                                            active='exact')],
                               fill=True,
                               pills=True)],
                      width=5,
                      style={'padding':'20px 0px'},
                      align='end')])

In [8]:
# County/State view dropdowns
dropdowns = dbc.Row([# Dropdowns 
                create_dropdown('Demographic', doptions, 'pop_selection'), 
                create_dropdown('Facility',    hoptions, 'fac_selection'),
                create_dropdown('Jurisdiction',joptions, 'jur_selection'),
                create_dropdown('Map Display', choptions,'cho_selection')],
                style={'padding':'0px 50px'})

In [9]:
# county/state view options
options = dbc.Row([
              create_radio('pop_stat', pct_disabled),
              create_radio('fac_stat', rad_foptions),
              create_checklist('scale_pop',rad_joptions),
              create_checklist('show_hosp',rad_choptions)],
              style={'padding':'0px 30px'})

In [10]:
# county/state scatter/choropleth
choropleth = dbc.Row([# plots
                 dcc.Graph(id='plot',figure={})],
                     style={'padding':'0px 45px'})

In [11]:
# Data sources page
citation = dbc.Col([
               html.P(),
               html.H5('Bureau of Economic Analysis'),
               html.A(f'GDP and Personal Income. {bea_url}. Accessed 31 May 2021',
                      href=bea_url),
               html.P(),
               html.H5('Center for Disease Control'),
               html.A(f'U.S. Small-area Life Expectancy Estimates Project. {cdc_url}. Accessed 31 May 2021',
                      href=cdc_url),
               html.P(),
               html.H5('Homeland Infrastructure Foundation-Level Data (HIFLD)'),
               html.A(f'Hospitals. {hifld_url}. Accessed 31 May 2021',
                      href=hifld_url),
               html.P(),
               html.H5('US Census'),
               html.A(f'Modified Race Data 2010. {census_url}. Accessed 31 May 2021',
                      href=census_url)],
               style={'padding':'0px 35px'})

In [12]:
# master layout
layout = html.Div([header,                          # Header with page tabs
                   dcc.Store(id='save_values'),     # Saves most recent column selections
                   dcc.Location(id="url"),          # Current tab
                   html.Div(id='scatter_display',   # Scatters and choropleth
                            hidden=False,
                            children=[dropdowns,    
                                      options,
                                      choropleth]),
                   html.Div(id='citation_display',  # Citations
                            hidden=True,
                            children=[citation])])    

# Plot

In [13]:
def plot_scatters(x_axis,y_axis,life_exp,income,pop_stat,
                  hovers,point_size,colors,
                  choropleth_column,locations,geo,
                  hospitals,show_hosp):
    '''Creates subplots of 3 scatterplots with a choropleth for either state or county data.
        Inputs:
            x_axis, y_axis, life_exp, income (arrays): axes for scatters
            pop_stat (str): if 'Percentage ' x-axis ticks will be formatted as pcts 
            hovers, point_size, colors (arrays): scatter customizations
            choropleth_column, locations (arrays): data for mapping choropleth
            geo (geojson or None): geojson if plotting county data, none for state data
            hospitals (geo dataframe): for plotting facility locations
            show_hosp (list): if longer than 1, hospital locations with be shown
        Returns:
            fig (plotly figure)            
    '''
    
    fig = make_subplots(rows=3, cols=3,
                        specs=[[{},{'rowspan':3,'colspan':2,'type':'choropleth'},None],
                              [{},None,None],
                              [{},None,None]],
                        shared_xaxes=True,
                        horizontal_spacing=0.01,
                        vertical_spacing=0.01)

    fig.add_trace(go.Scatter(x=x_axis, 
                             y=y_axis,
                             text=hovers,
                             hovertemplate='%{text}<br>%{x}<br>%{y}<extra></extra>',
                             opacity=0.75,
                             mode='markers',
                             showlegend=False,
                             marker=dict(size=point_size,
                                         color=colors)),
                  row=1,col=1)

    fig.add_trace(go.Scatter(x=x_axis, 
                             y=life_exp,
                             text=hovers,
                             hovertemplate='%{text}<br>%{x}<br>%{y}<extra></extra>',
                             opacity=0.75,
                             mode='markers',
                             showlegend=False,
                             marker=dict(size=point_size,
                                         color=colors)),
                  row=2,col=1)

    fig.add_trace(go.Scatter(x=x_axis, 
                             y=income,
                             text=hovers,
                             hovertemplate='%{text}<br>%{x}<br>Income: %{y}<extra></extra>',
                             opacity=0.75,
                             mode='markers',
                             showlegend=False,
                             marker=dict(size=point_size,
                                         color=colors)),
                  row=3,col=1)

    fig.add_trace(go.Choropleth(geojson=geo, 
                                locations=locations,
                                locationmode='USA-states' if geo==None else None,
                                z=choropleth_column,
                                colorscale="Viridis", 
                                zmin=choropleth_column.min(), 
                                zmax=choropleth_column.max(),
                                text=hovers,
                                hovertemplate='%{text}<br>%{z}<extra></extra>',
                                showscale=False),
                  row=1,col=2)
    
    if len(show_hosp)!=1:
        fig.add_trace(go.Scattergeo(locationmode='USA-states',
                                    lon=hospitals.geometry.x,
                                    lat=hospitals.geometry.y,
                                    text=hospitals.hovers,
                                    mode='markers',
                                    hovertemplate='%{text}<extra></extra>',
                                    marker_color='red',
                                    marker = dict(size = 4,
                                                  opacity = 0.8,
                                                  line = dict(width=0.5,
                                                              color='white')),
                                    showlegend=False),
                 row=1,col=2)
    
    fig.update_geos(scope='usa',
                    row=1,col=2)

    fig['layout']['yaxis' ]['title'] , fig['layout']['xaxis3']['title'] = y_axis.name , x_axis.name
    fig['layout']['yaxis2']['title'] , fig['layout']['yaxis3']['title'] = 'Life Expectancy' , 'Income'
    
    if pop_stat == 'Percentage ':
        fig.layout.xaxis3.tickformat = ',.0%'

    fig.update_layout(width=1800,height=1000,
                      margin=dict(t=10,b=10,l=50,r=10),
                      template="plotly_dark",
                      hovermode='closest')

    return fig 

# Callbacks

In [14]:
def check_button(triggered,data):
    '''Determines last click on dashboard and changes default plotting columns accordingly.
        Inputs:
            triggered (dash.callback_context.trigerred): latest click
            data (list): cached list of columns, initialized in first call and updated with subsequent
        Returns:
            data (list): list of 4 columns to be plotted
    '''
    if data is None:
        return ['Total Population','Total Population','All Facilities','US']
    if triggered: # Assign last click to corresponding variable
        t = triggered[0]["prop_id"].split(".")[0].replace('_',' ').replace('+','/')
        data[0] = t[:-1] if t[-1]=='2' else data[0] 
        data[1] = t[:-1] if t[-1]=='3' else data[1]
        data[2] = t[:-1] if t[-1]=='4' else data[2]
        data[3] = t[:-1] if t[-1]=='5' else data[3]
    return data

In [15]:
app = dash.Dash(__name__,
                external_stylesheets=[dbc.themes.CYBORG])

app.layout = layout

@app.callback(
       [Output('save_values','data'),                          # caches last selection
        Output('pop_stat','value'),                            # resets pct to count if Total Population selected
        Output('pop_stat','options'),                          # disables pct if Total Population selected
        Output('scatter_display','hidden'),                    # toggles plot and options visibility
        Output('citation_display','hidden'),                   # toggles citiations visibility
        Output('pop_selection',component_property='children'), # display selected race
        Output('fac_selection',component_property='children'), # display selected facility
        Output('jur_selection',component_property='children'), # display selected jurisdiction
        Output('cho_selection',component_property='children'), # display selected choropleth column
        Output('plot',component_property='figure')],           # plot
         
       [Input('save_values','data'),                           # cached defaults/latest selection
        Input('url','pathname'),                               # current page
        Input('pop_stat','value'),                             # pop count,pct,density
        Input('fac_stat','value'),                             # facilities,per capita,density
        Input('scale_pop','value'),                            # scale scatter points
        Input('show_hosp','value')] +                          # show hospitals on choropleth
       [Input(f"{i.replace(' ','_').replace('/','+')}2",'n_clicks') for i in map_col] +        # choropleth 
       [Input(f"{i.replace(' ','_').replace('/','+')}3",'n_clicks') for i in races]   +        # demographic
       [Input(f"{i.replace(' ','_').replace('/','+')}4",'n_clicks') for i in hospital_types] + # facility
       [Input(f"{i.replace(' ','_').replace('/','+')}5",'n_clicks') for i in alljurs])         # jurisdiction

def update_graph(data, path_name, pop_stat, fac_stat, scale_pop, show_hosp,              
                 # choropleth options
                 Multiracial2,Native_Hawaiian1Pacific_Islander2,American1Alaskan_Native2,Asian2,
                 Black2,White2,Hispanic2,Total_Population2,     
                 Percentage_Multiracial2,Percentage_Native_Hawaiian1Pacific_Islander2,
                 Percentage_American1Alaskan_Native2,Percentage_Asian2,Percentage_Black2,Percentage_White2,
                 Percentage_Hispanic2, 
                 Multiracial_Population_per_Square_km2,Native_Hawaiian1Pacific_Islander_Population_per_Square_km2,
                 American1Alaskan_Native_Population_per_Square_km2,Asian_Population_per_Square_km2, 
                 Black_Population_per_Square_km2,White_Population_per_Square_km2,
                 Hispanic_Population_per_Square_km2,Total_Population_per_Square_km2,
                 Children2,Chronic_Disease2,Critical_Access2,General_Acute_Care2,Long_Term_Care2, 
                 Military2,Psychiatric2,Rehabilitation2,Special2,Women2,All_Facilities2, 
                 General_Acute_Care_Facilities_per_100k2,Psychiatric_Facilities_per_100k2, 
                 Children_Facilities_per_100k2,Long_Term_Care_Facilities_per_100k2,
                 Critical_Access_Facilities_per_100k2,Rehabilitation_Facilities_per_100k2, 
                 Military_Facilities_per_100k2,Women_Facilities_per_100k2,Special_Facilities_per_100k2,
                 Chronic_Disease_Facilities_per_100k2,All_Facilities_per_100k2,
                 General_Acute_Care_Facilities_per_Square_km2,Psychiatric_Facilities_per_Square_km2,
                 Children_Facilities_per_Square_km2,Long_Term_Care_Facilities_per_Square_km2,
                 Critical_Access_Facilities_per_Square_km2,Rehabilitation_Facilities_per_Square_km2,
                 Military_Facilities_per_Square_km2,Women_Facilities_per_Square_km2,
                 Special_Facilities_per_Square_km2,Chronic_Disease_Facilities_per_Square_km2,
                 All_Facilities_per_Square_km2,            
                 Average_Income2,Average_Life_Expectancy2,
                 # Demographic options
                 Multiracial3,Native_Hawaiian1Pacific_Islander3,American1Alaskan_Native3,Asian3,
                 Black3,White3,Hispanic3,Total_Population3,
                 # Facility options
                 Children4,Chronic_Disease4,Critical_Access4,General_Acute_Care4,Long_Term_Care4, 
                 Military4,Psychiatric4,Rehabilitation4,Special4,Women4,All_Facilities4,
                 # Jurisdictions
                 US5,West5,Southwest5,Midwest5,Southeast5,Northeast5,
                 Alabama5,Alaska5,Arizona5,Arkansas5,California5,Colorado5,Connecticut5,Delaware5,
                 District_of_Columbia5,Florida5,Georgia5,Hawaii5,Idaho5,Illinois5,Indiana,Iowa,Kansas5,Kentucky,
                 Louisiana5,Maine5,Maryland5,Massachusetts5,Michigan5,Minnesota5,Mississippi5,Missouri5,
                 Montana5,Nebraska5,Nevada5,New_Hampshire5,New_Jersey5,New_Mexico5,New_York,
                 North_Carolina5,North_Dakota5,Ohio5,Oklahoma5,Oregon5,Pennsylvania5,Rhode_Island5,
                 South_Carolina5,South_Dakota5,Tennessee5,Texas5,Utah5,Vermont5,Virginia5,Washington5,
                 West_Virginia5,Wisconsin5,Wyoming5):
    '''Creates dashboard of healthcare access data.
        Inputs:
            data (list): cached list of columns being plotted, updated and saved with each callback
            path_name (str): current url extension
            pop_stat (str): determines if scatter x-axis to be counts, pct, or density
            fac_stat (str): determines if scatter y-axis to be counts, per 100k, or density
            scale_pop (string): determines if scatter points will be scaled by population
            show_hosp (list): determines if hospital locations will be shown on choropleth
            <column>2 (str): inputs from choropleth dropdown, determines choropleth display
            <column>3 (str): inputs from demographic dropdown, determines race plotted by scatters
            <column>4 (str): inputs from facilities dropdown, determines facility type plotted by scatters
            <column>5 (str): inputs from jurisdiction dropdown, determines location plotted by scatters
        Returns:
            data (list): updated list of columns being plotted, to be cached for future callbacks
            pop_stat (str): changes selected population stat to counts if Total Population selected
            disable_button (dictionary): disables pct as an option from population statistics if
                Total Population selected.
            hide_scatter/not hide_scatter (boolean): hides plots/dropdown if citation page selected and 
                vice versa.
            x_axis, y_axis, filter_state, choropleth_column (strings): update dashboard with selected columns
            plot_scatters(args) (plotly figure): updated plot 
    '''

    choropleth_column,x_axis,y_axis,filter_state = data = check_button(dash.callback_context.triggered,data) 
      
    pop_stat = '' if (x_axis=='Total Population' and pop_stat =='Percentage ') else pop_stat
    x_axis_stat = (pop_stat+x_axis) if pop_stat=='Percentage ' else (x_axis+pop_stat) 
        
    y_axis_stat = 'All'+fac_stat if (y_axis=='All Facilities' and fac_stat!='') else y_axis+fac_stat
    
    df = S.copy() if path_name=='/' else H.copy() # pull either state or county-level dataframe
    P = df.copy() # Maintain unfiltered copy for choropleth
    
    if filter_state!='US':
        df = df[df.Region==filter_state] if filter_state in regions else df[df.State==filter_state]
        
    geo = None if path_name=='/' else counties_json
    point_size = 7 if len(scale_pop)==1 else df.sizes # update scatter size
    
    hide_scatter = False if path_name!='/data' else True
    disable_button = pct_disabled if x_axis=='Total Population' else pct_active

        
    return data, pop_stat, disable_button, hide_scatter, not hide_scatter,\
           x_axis, y_axis, filter_state, choropleth_column, \
           plot_scatters(df[x_axis_stat],df[y_axis_stat],
                         df['Average Life Expectancy'],
                         df["Average Income"],
                         pop_stat,
                         df.hovers,
                         point_size,
                         df.colors,
                         P[choropleth_column],
                         P.index,
                         geo,
                         hospitals,
                         show_hosp)


In [16]:
app.run_server(debug=True,use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
